# Part II - VacationPy
#### By raulfloresp
----

Plan future vacations - Use jupyter-gmaps and the Google Places API for this part of the assignment.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Google developer API key
from config_google import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
# Set path for file
csv_path = os.path.join("Output", "cities.csv")

# Import the data into a Pandas DataFrame
cities_df = pd.read_csv(csv_path)
cities_df

,City,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Latitude,Longitude,Country
0,Caconda,52.11,58,7,3.89,-13.73,15.07,AO
1,Punta Arenas,39.09,86,75,4.70,-53.15,-70.92,CL
2,Ushuaia,39.20,93,75,4.70,-54.80,-68.30,AR
3,Ndélé,75.49,80,87,4.72,8.41,20.65,CF
4,Synya,64.33,62,98,7.94,65.37,58.04,RU
...,...,...,...,...,...,...,...,...
549,Jiblah,72.14,47,57,3.09,13.92,44.15,YE
550,Bad Liebenzell,50.74,100,75,5.82,48.77,8.73,DE
551,Kurchum,58.91,43,27,11.74,48.57,83.65,KZ
552,Iquique,59.00,82,61,1.34,-20.22,-70.14,CL


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Configure gmaps
gmaps.configure(api_key=gkey)

In [16]:
# Use the Lat and Lng as locations and Humidity as the weight
locations = cities_df[["Latitude", "Longitude"]].astype(float)
humidity = cities_df["Humidity (%)"].astype(float)

In [17]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

#### Weather Conditions
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don’t contain all three conditions.

In [7]:
# Look for null values
cities_df.isnull().sum()
#No null values were found

City                0
Temperature (F)     0
Humidity (%)        0
Cloudiness (%)      0
Wind Speed (mph)    0
Latitude            0
Longitude           0
Country             3
dtype: int64

In [8]:
cities_df.count()

City                554
Temperature (F)     554
Humidity (%)        554
Cloudiness (%)      554
Wind Speed (mph)    554
Latitude            554
Longitude           554
Country             551
dtype: int64

In [9]:
# Building a new data frame for cities that meet weather conditions

# Narrow cities with temperature bellow 80 degrees F
cities_80_degrees = cities_df[cities_df["Temperature (F)"] < 80]

# Narrow cities with temperature above 70 degrees F
cities_70_degrees = cities_80_degrees[cities_80_degrees["Temperature (F)"] > 70]

# Narrow cities with Wind speed less than 10 mph.
cities_10mph_wind = cities_70_degrees[cities_70_degrees["Wind Speed (mph)"] < 10]

# Narrow cities with zero cloudiness
cities_zero_cloud = cities_10mph_wind[cities_10mph_wind["Cloudiness (%)"] == 0]

# Set Index
my_vacation_cities = cities_zero_cloud.reset_index()
del my_vacation_cities["index"]
my_vacation_cities.head()

,City,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Latitude,Longitude,Country
0,Suntar,77.40,21,0,6.35,62.14,117.63,RU
1,Yatou,75.16,43,0,5.99,37.15,122.38,CN
2,Mersin Province,78.80,50,0,5.82,36.86,34.65,TR
3,Aleksandrov Gay,71.33,53,0,8.16,50.15,48.57,RU
4,Betioky,71.96,59,0,9.22,-23.72,44.38,MG


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Store into variable named hotel_df
hotel_df = []

# Add a "Hotel Name" column to the DataFrame.
for city in range(len(my_vacation_cities["City"])):

    lat = my_vacation_cities.loc[city]["Latitude"]
    lng = my_vacation_cities.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000, #Set parameters to search for hotels with 5000 meters.
        "key": gkey
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotel_df.append(hotel_response["results"][0]["name"])
    except:
        hotel_df.append("Hotel not found")

# Build the Dataframe with the nearest hotel
my_vacation_cities["Hotel Name"] = hotel_df
hotel_df = my_vacation_cities
hotel_df

,City,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Latitude,Longitude,Country,Hotel Name
0,Suntar,77.40,21,0,6.35,62.14,117.63,RU,Gostevoy Dom
1,Yatou,75.16,43,0,5.99,37.15,122.38,CN,GreenTree Hotel Rongcheng Bus Station
2,Mersin Province,78.80,50,0,5.82,36.86,34.65,TR,Nostalgia Boutique Hotel
3,Aleksandrov Gay,71.33,53,0,8.16,50.15,48.57,RU,Hotel not found
4,Betioky,71.96,59,0,9.22,-23.72,44.38,MG,Bungalows Acacias Hotel
5,Lentekhi,72.16,31,0,3.85,42.79,42.73,GE,Konstanta Hotel
6,Saint-Paul,71.91,57,0,2.24,-21.01,55.27,RE,Boucan Canot
7,Makhinjauri,71.60,56,0,8.05,41.67,41.69,GE,Batumi World Palace
8,Lobito,73.69,69,0,3.38,-12.35,13.55,AO,Casa Rosa Hotel Residence
9,Cosalá,78.01,88,0,1.01,24.41,-106.69,MX,Letras Cosala


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

### End of excersise